# Xmas Bayesian Optimisation Guessing Game

This notebook implements a xmas-themed guessing game to try and find the maximum of a black-box function

In [1]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import altair as alt
import blackbox as bb
from importlib import reload
reload(bb);

## 1-dimensional input
Create a scalar valued black-box function of a 1-dimensional input and a widget to control which point to evaluate at.

In [2]:
blackbox1 = bb.GPBlackBox(ndim=1)
x0 = widgets.FloatSlider(
    value=0,
    min=bb.DOMAIN_MIN,
    max=bb.DOMAIN_MAX,
    step=0.01,
    description='x0:',
    readout_format='.2f',
)

Choose which input point (x0) to evaluate the function at:

In [3]:
display(x0)

FloatSlider(value=0.0, description='x0:', max=1.0, min=-1.0, step=0.01)

Evaluate the function and plot all the evaluations so far:

In [4]:
y = blackbox1([x0.value])[0][0]
print(f'Evaluated black box at {x0.value}; result={y}')
blackbox1.plot_xy()

Evaluated black box at 0.0; result=0.27897901659326874


alt.Chart(...)

If you have finished evaluating the function at different points and you are confident where the maximum is, you can make a guess before executing the cells below.

Now show the function as a line and the noisy data we received as evaluations of it:

In [5]:
f1 = blackbox1.sample_f(100)
chart1f = (
    alt.Chart(f1)
    .mark_line()
    .encode(x='x0', y='f'))
chart1y = blackbox1.plot_xy()
chart1 = alt.layer(chart1y, chart1f)
chart1

alt.LayerChart(...)

## 2-dimensional input
Create a scalar valued black-box function of a 2-dimensional input and widgets to control which point to evaluate at.

In [6]:
blackbox2 = bb.GPBlackBox(ndim=2)
x0 = widgets.FloatSlider(
    value=0,
    min=bb.DOMAIN_MIN,
    max=bb.DOMAIN_MAX,
    step=0.01,
    description='x0:',
    readout_format='.2f',
)
x1 = widgets.FloatSlider(
    value=0,
    min=bb.DOMAIN_MIN,
    max=bb.DOMAIN_MAX,
    step=0.01,
    description='x1:',
    readout_format='.2f',
)
w = widgets.Box([x0, x1])

Choose which input point (x0, x1) to evaluate the function at:

In [7]:
display(w)

Box(children=(FloatSlider(value=0.0, description='x0:', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, …

Evaluate the function and plot all the evaluations so far:

In [8]:
y = blackbox2([x0.value, x1.value])[0][0]
print(f'Evaluated black box at ({x0.value}, {x1.value}); result={y}')
blackbox2.plot_xy()

Evaluated black box at (0.0, 0.0); result=-0.10281844879401922


alt.Chart(...)

If you have finished evaluating the function at different points and you are confident where the maximum is, you can make a guess before executing the cells below.

Now show the underlying function f (without noise) as a heatmap and the noisy data we received as evaluations of it:

In [9]:
f2 = blackbox2.sample_f(45)
chart2f = (
    alt.Chart(f2)
    .mark_square(size=30)
    .encode(x=alt.X('x0:Q', scale=alt.Scale(domain=bb.DOMAIN)),
            y=alt.X('x1:Q', scale=alt.Scale(domain=bb.DOMAIN)),
            color=alt.Color('f:Q', scale=alt.Scale(scheme=bb.COLOURSCHEME, domainMid=0))))
chart2y = blackbox2.plot_xy()
chart2 = alt.layer(chart2f, chart2y)
chart2

alt.LayerChart(...)